# chengshuai — HFO 群体事件可视化

## 架构原则（2026-01-16 重构）

> **本 notebook 只负责读取中间结果并可视化，不做任何复杂计算。**

**数据流**：
```
*_groupAnalysis.npz (质心/TF质心/lag/rank)
*_envCache.npz (envelope + 带通信号)       →  本Notebook (只读取+画图)  →  matplotlib
*_packedTimes.npy (事件窗口)
```

**可视化内容**：
1. Fig1：带通波形 raster（拼接事件窗口）
2. Fig2：TF 功率背景 + 质心点 + 传播路径
3. 热图：channels × events 的 energy/rank/lag
4. 统计：lag 分布、TF 质心分布、通道参与率

In [ ]:
import sys
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import importlib

sys.path.insert(0, '/home/honglab/leijiaxin/HFOsp/')

# Visualization-only imports (读取中间结果 + 画图)
from src.group_event_analysis import (
    load_group_analysis_results,
    load_envelope_cache,
    compute_and_save_group_analysis,
)
from src.preprocessing import save_raw_cache, load_raw_cache
import src.visualization as visualization
importlib.reload(visualization)
from src.visualization import (
    plot_paper_fig1_bandpassed_traces,
    plot_paper_fig2_normalized_spectrogram,
    plot_lag_heatmaps_from_group_analysis,
    plot_lag_statistics,
    plot_tf_centroid_statistics,
    plot_bandpassed_segment_from_cache,
    plot_channel_waveform_and_envelope_from_cache,
    build_event_spans_from_packed_times,
    plot_channel_lag_distributions,
    plot_channel_centroid_distributions,
    plot_raw_segment_from_cache,
)

# Basic visualization settings
BASIC_START_SEC = 0.0
BASIC_DURATION_SEC = 10.0
SHAFT = 'K'
CHANNEL_LIST = ['K1-K2', 'K2-K3', 'K3-K4']
SINGLE_CHANNEL = 'K3-K4'
USE_RAW = True

# Configuration
PATIENT = 'chengshuai'
RECORD = 'FC10477Q'
DATA_ROOT = Path('/mnt/yuquan_data/yuquan_24h_edf')

EDF_PATH = DATA_ROOT / PATIENT / f'{RECORD}.edf'
GPU_NPZ = DATA_ROOT / PATIENT / f'{RECORD}_gpu.npz'
PACKED_TIMES = DATA_ROOT / PATIENT / f'{RECORD}_packedTimes.npy'
LAGPAT_NPZ = DATA_ROOT / PATIENT / f'{RECORD}_lagPat.npz'
GROUP_ANALYSIS_PATH = DATA_ROOT / PATIENT / f'{RECORD}_groupAnalysis.npz'
ENV_CACHE_PATH = DATA_ROOT / PATIENT / f'{RECORD}_envCache_ripple_bipolar.npz'
RAW_CACHE_PATH = DATA_ROOT / PATIENT / f'{RECORD}_rawCache_none.npz'

# Visualization settings
EVENT_INDICES = list(range(30))  # First 30 events

%matplotlib inline
plt.rcParams['figure.dpi'] = 300

ImportError: cannot import name 'plot_bandpassed_segment_from_cache' from 'src.visualization' (/home/honglab/leijiaxin/HFOsp/src/visualization.py)

## 1. 检查/生成中间结果

如果 `*_groupAnalysis.npz` 和 `*_envCache.npz` 不存在，则调用 `compute_and_save_group_analysis()` 生成。

In [ ]:
print('=== 检查中间结果 ===')
print(f'Group analysis exists: {GROUP_ANALYSIS_PATH.exists()}')
print(f'Env cache exists: {ENV_CACHE_PATH.exists()}')
print(f'Raw cache exists: {RAW_CACHE_PATH.exists()}')

if not GROUP_ANALYSIS_PATH.exists() or not ENV_CACHE_PATH.exists():
    print('\n生成中间结果...')
    out_paths = compute_and_save_group_analysis(
        edf_path=str(EDF_PATH),
        output_dir=str(DATA_ROOT / PATIENT),
        output_prefix=RECORD,
        packed_times_path=str(PACKED_TIMES),
        gpu_npz_path=str(GPU_NPZ),
        band='ripple',
        reference='bipolar',
        alias_bipolar_to_left=True,
        crop_seconds=120.0,
        use_gpu=True,
        save_env_cache=True,
    )
    print('✓ 生成完成')
else:
    print('✓ 中间结果已存在，跳过生成')

if USE_RAW and not RAW_CACHE_PATH.exists():
    print('\n生成 rawCache (no filter/resample)...')
    raw_path = save_raw_cache(
        edf_path=str(EDF_PATH),
        output_dir=str(DATA_ROOT / PATIENT),
        output_prefix=RECORD,
        reference='none',
        crop_seconds=120.0,
    )
    print(f'✓ rawCache created: {raw_path}')
elif USE_RAW:
    print('✓ rawCache 已存在，跳过生成')

## 2. 加载中间结果

加载预计算的 group analysis 结果和 envelope cache。

In [ ]:
# Load pre-computed group analysis results
ga = load_group_analysis_results(str(GROUP_ANALYSIS_PATH))
cache = load_envelope_cache(str(ENV_CACHE_PATH))

print('=== Group Analysis ===')  
print(f'  n_channels: {ga["n_channels"]}')
print(f'  n_events: {ga["n_events"]}')
print(f'  band: {ga["band"]}')
print(f'  sfreq: {ga["sfreq"]} Hz')

print('\n=== Envelope Cache ===')
print(f'  env shape: {cache["env"].shape}')
print(f'  x_band available: {cache["x_band"] is not None}')

# Load core channels from lagPat reference
lag_ref = np.load(LAGPAT_NPZ, allow_pickle=True)
CORE_CHANNELS = [str(x) for x in lag_ref['chnNames'].tolist()]
print(f'\n=== Core Channels (n={len(CORE_CHANNELS)}) ===')
print(f'  {CORE_CHANNELS[:10]}...')

## 3. 基础可视化：全通道 / 电极 / 通道列表

基于 `*_envCache.npz` 的带通波形与 envelope，并用群体事件窗口进行红色 shading。

In [ ]:
# Build event spans for shading
event_spans = build_event_spans_from_packed_times(
    str(PACKED_TIMES),
    start_sec=BASIC_START_SEC,
    end_sec=BASIC_START_SEC + BASIC_DURATION_SEC,
    max_events=200,
    color="#e94560",
    alpha=0.15,
)

# 3.1 Raw Cache 全通道波形 (来自 preprocessing.py)
if USE_RAW:
    fig_raw = plot_raw_segment_from_cache(
        raw_cache_npz_path=str(RAW_CACHE_PATH),
        packed_times_path=str(PACKED_TIMES),
        start_sec=BASIC_START_SEC,
        duration_sec=BASIC_DURATION_SEC,
        channels='all',
        color_by_shaft=True,
        event_alpha=0.15,
        max_events=200,
        figsize=(16, 12),
        title=f"{PATIENT}/{RECORD} — Raw Cache (all channels)",
    )
    plt.show()

# 3.2 全通道带通波形 (stacked)
fig_all = plot_bandpassed_segment_from_cache(
    cache_npz_path=str(ENV_CACHE_PATH),
    packed_times_path=str(PACKED_TIMES),
    start_sec=BASIC_START_SEC,
    duration_sec=BASIC_DURATION_SEC,
    channels='all',
    mode='bandpassed',
    color_by_shaft=True,
    event_alpha=0.15,
    max_events=200,
    figsize=(16, 12),
    title=f"{PATIENT}/{RECORD} — Bandpassed (all channels)",
)
plt.show()

# 3.3 单电极串 (shaft) 带通波形
fig_shaft = plot_bandpassed_segment_from_cache(
    cache_npz_path=str(ENV_CACHE_PATH),
    packed_times_path=str(PACKED_TIMES),
    start_sec=BASIC_START_SEC,
    duration_sec=BASIC_DURATION_SEC,
    channels=[ch for ch in cache['ch_names'] if str(ch).startswith(SHAFT)],
    mode='bandpassed',
    color_by_shaft=True,
    event_alpha=0.15,
    max_events=200,
    figsize=(12, 8),
    title=f"{PATIENT}/{RECORD} — Shaft {SHAFT}",
)
plt.show()

# 3.4 指定通道列表
fig_list = plot_bandpassed_segment_from_cache(
    cache_npz_path=str(ENV_CACHE_PATH),
    packed_times_path=str(PACKED_TIMES),
    start_sec=BASIC_START_SEC,
    duration_sec=BASIC_DURATION_SEC,
    channels=CHANNEL_LIST,
    mode='bandpassed',
    color_by_shaft=True,
    event_alpha=0.15,
    max_events=200,
    figsize=(12, 6),
    title=f"{PATIENT}/{RECORD} — Selected channels",
)
plt.show()

# 3.5 单通道：波形 vs Envelope（左右对照）
fig_we = plot_channel_waveform_and_envelope_from_cache(
    cache_npz_path=str(ENV_CACHE_PATH),
    packed_times_path=str(PACKED_TIMES),
    channel=SINGLE_CHANNEL,
    start_sec=BASIC_START_SEC,
    duration_sec=2.0,
    event_alpha=0.2,
    max_events=200,
    figsize=(12, 4),
)
plt.show()

## 4. Fig1: 带通波形 Raster

显示拼接的事件窗口中的 80-250Hz 带通波形。

In [ ]:
fig1 = plot_paper_fig1_bandpassed_traces(
    cache_npz_path=str(ENV_CACHE_PATH),
    packed_times_path=str(PACKED_TIMES),
    channel_order=CORE_CHANNELS,
    event_indices=EVENT_INDICES,
    max_events=len(EVENT_INDICES),
    figsize=(16, 10),
)
plt.suptitle(f"{PATIENT}/{RECORD} — Bandpassed Traces (80-250Hz)", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 5. Fig2: TF 功率 + 质心路径

STFT 归一化功率背景 + 每事件的 TF 质心点 + 传播路径。

**注意**：现在从 `*_groupAnalysis.npz` 读取预计算的 TF 质心，不再内部计算。

In [ ]:
fig2 = plot_paper_fig2_normalized_spectrogram(
    cache_npz_path=str(ENV_CACHE_PATH),
    packed_times_path=str(PACKED_TIMES),
    detections_npz_path=str(GPU_NPZ),
    group_analysis_npz_path=str(GROUP_ANALYSIS_PATH),
    channel_order=CORE_CHANNELS,
    event_indices=EVENT_INDICES,
    max_events=len(EVENT_INDICES),
    freq_band=(80.0, 250.0),
    centroid_marker_size=60.0,
    figsize=(16, 10),
)
plt.suptitle(f"{PATIENT}/{RECORD} — TF Power + Centroid Paths", fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 6. 热图：Energy / Rank / Lag

直接从 `*_groupAnalysis.npz` 读取预计算的结果。

In [ ]:
figE, figR, figL = plot_lag_heatmaps_from_group_analysis(
    group_analysis_npz=str(GROUP_ANALYSIS_PATH),
    env_cache_npz=str(ENV_CACHE_PATH),
    packed_times_npy=str(PACKED_TIMES),
    channel_names=CORE_CHANNELS,
    max_events=100,
    figsize=(14, 4),
)
plt.show()


## 7. 统计分析：Lag / Rank / Participation

从 `*_groupAnalysis.npz` 读取预计算结果进行统计。

In [ ]:
fig_stats = plot_lag_statistics(
    group_analysis_npz=str(GROUP_ANALYSIS_PATH),
    patient_id=PATIENT,
    record_id=RECORD,
    figsize=(15, 4),
)
plt.show()


## 8. 每通道 Lag/质心 分布（自上而下）

参考图示：每个通道一条分布，红线连接每通道的质心。

In [ ]:
fig_lag_ridge = plot_channel_lag_distributions(
    group_analysis_npz=str(GROUP_ANALYSIS_PATH),
    channel_order=CORE_CHANNELS,
    bins=40,
    figsize=(6, 10),
    title=f"{PATIENT}/{RECORD} — Per-channel lag distributions",
)
plt.show()

fig_centroid_ridge = plot_channel_centroid_distributions(
    group_analysis_npz=str(GROUP_ANALYSIS_PATH),
    channel_order=CORE_CHANNELS,
    bins=40,
    figsize=(6, 10),
    title=f"{PATIENT}/{RECORD} — Per-channel centroid time distributions",
)
plt.show()

## 9. TF 质心统计

从 `*_groupAnalysis.npz` 读取预计算的 TF 质心结果。

In [ ]:
fig_tf = plot_tf_centroid_statistics(
    group_analysis_npz=str(GROUP_ANALYSIS_PATH),
    patient_id=PATIENT,
    record_id=RECORD,
    figsize=(12, 4),
)
if fig_tf is not None:
    plt.show()
else:
    print('TF centroids not available in groupAnalysis results')


---

## 附录：切换患者/记录

只需修改 Cell 1 中的 `PATIENT`/`RECORD` 配置，重新运行即可。

所有可视化函数都从 `src/visualization.py` 调用，不做任何复杂计算。